# Eflux2 applied to *P. putida*
This notebook is based on this [*A. niger* example](https://github.com/AgileBioFoundry/AspergillusQ4Milestone/blob/main/notebooks/Eflux4A.niger.ipynb).

In [1]:
import os
import sys

import cobra
import pandas as pd
# pd.set_option('display.max_rows', None)

import numpy as np

from optlang import gurobi_interface
from optlang.symbolics import add

import cplex

In [2]:
os.chdir('../scripts/')
from eflux2 import EFlux2
os.chdir('../notebooks')

`../parsed-data/core80_reactions-genes.csv` lists the association between reactions and genes.

Make sure that the model you import has the relevant genes associated with each reaction.

In [3]:
model = cobra.io.load_json_model('../models/iJN1463_JS.json')
model.solver = 'gurobi'
N = cobra.util.create_stoichiometric_matrix(model)
model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-23


Name,iJN1463
Memory address,2341f2a9f40
Number of metabolites,671
Number of reactions,637
Number of genes,596
Number of groups,0
Objective expression,1.0*RgPAL - 1.0*RgPAL_reverse_f41ed
Compartments,"cytosol, periplasm, extracellular space"


## Import transcriptomics data


In [4]:
intensities = pd.read_csv('../data/putida_RNAseq_data.csv').iloc[: , 1:]
intensities.describe()

,Value
count,228575.000000
mean,547.688969
std,8478.086255
min,0.000000
25%,39.000000
50%,104.000000
75%,282.000000
max,776103.000000


Since the range of the values are quite large, we perform additive smoothing (i.e. Laplace smoothing)


In [5]:
ADD = 1.1
intensities['logValues'] = np.log(intensities['Value'].astype('float') + ADD)
intensities.describe()

,Value,logValues
count,228575.000000,228575.000000
mean,547.688969,4.644346
std,8478.086255,1.583710
min,0.000000,0.095310
25%,39.000000,3.691376
50%,104.000000,4.654912
75%,282.000000,5.645800
max,776103.000000,13.562042


In [6]:
[ii for ii in set([i for i in intensities['genes'] if 'PP' not in i]) if ii not in model.genes]

['AroL',
 'MCP-SoxS',
 'gentamycin-3-acetyltransferase',
 'AroG-fbr',
 'mob',
 'dCas9',
 'pBBR1 rep']

In [7]:
# attributing the nonCRISPR genes to their respective reactions
model.reactions.DDPA.gene_reaction_rule = '( PP_1866 and PP_2324 and PP_3080 and AroG-fbr )'
model.reactions.SHKK.gene_reaction_rule = '( PP_5079 and AroL )'

In [8]:
cleaned_data = pd.pivot_table(intensities, values='logValues', index='genes', columns='strains').round(4)
cleaned_data.head(10)

strains,4scRNA control,CRISPRa-PP_1075,CRISPRa-PP_1830,CRISPRa-PP_2168,CRISPRa-PP_2329,CRISPRa-PP_4169,CRISPRa-PP_4965,CRISPRa-PP_5079,CRISPRi-PP_0059,CRISPRi-PP_0289,...,CRISPRi-PP_4297,CRISPRi-PP_4678,CRISPRi-PP_4725,CRISPRi-PP_4822,CRISPRi-PP_4823,CRISPRi-PP_5015,CRISPRi-PP_5128,CRISPRi-PP_5155,CRISPRi-PP_5336-5335,pACA production 3 scRNA positive control
genes,,,,,,,,,,,,,,,,,,,,,
AroG-fbr,10.2037,9.6915,10.2397,10.0788,9.0878,10.1344,10.2923,10.0395,9.7231,10.6007,...,10.0631,9.9279,9.3185,8.5585,8.5647,9.9541,10.0376,8.5274,8.5698,9.7556
AroL,8.4262,7.7459,8.2175,7.8517,7.2815,7.8835,7.9519,8.0017,7.9916,8.3540,...,8.0904,8.0530,7.7134,6.9208,6.9968,8.0784,7.9445,6.8481,7.0131,7.6765
MCP-SoxS,6.3904,6.7857,7.4043,7.2690,7.1453,7.0918,7.2464,7.1763,7.5050,8.2207,...,7.1648,6.8201,7.4025,6.8938,6.9461,8.0193,7.1067,6.7276,6.8618,7.1740
PP_0001,5.2988,5.5179,5.2938,5.3476,5.5495,5.3380,5.4557,5.3804,5.7624,6.1593,...,5.2476,5.0882,5.4935,4.8048,5.1990,5.6941,4.8836,5.3088,5.1654,5.5801
PP_0002,5.0046,5.4165,5.1305,5.3986,5.4254,5.0505,5.4254,5.0113,5.6316,5.8233,...,5.4076,5.2155,5.4254,4.8683,5.1066,5.5495,5.1654,4.9352,4.9843,5.5219
PP_0003,4.5860,4.5654,4.8911,4.7104,4.6357,4.1287,4.7194,4.5009,4.8606,4.7283,...,4.4671,4.3451,4.7715,4.0091,4.4555,4.9207,4.4898,4.3320,4.1759,4.7458
PP_0004,6.8629,7.1165,6.9557,6.9432,7.2241,6.9538,7.0140,7.0336,7.2725,7.1960,...,7.1042,6.8003,7.1157,6.6175,6.6872,7.1663,7.0185,6.8470,6.5696,7.2094
PP_0005,6.2617,6.0687,6.0405,5.9842,6.0429,6.0847,6.0938,5.7465,5.8639,5.6702,...,5.6563,5.7841,5.5138,5.7108,5.9138,5.7686,5.7749,5.6702,5.6839,6.1423
PP_0006,6.4601,6.6491,6.4054,6.3046,6.9168,6.6696,6.6162,6.7812,7.0519,7.3032,...,6.7880,6.3718,6.5059,6.3684,6.1740,6.6295,6.3511,6.3046,6.2540,6.6569


In [9]:
# find max of each row
cleaned_data.max(axis=1)
# normalize each row by dividing by the max possible value
norm_data = cleaned_data.divide(cleaned_data.max(axis=1), axis=0)
# multiply by glucose intake
    # model.metabolites.glc__D_e.summary()
    # norm_data = norm_data * 6

norm_data.head(10)

strains,4scRNA control,CRISPRa-PP_1075,CRISPRa-PP_1830,CRISPRa-PP_2168,CRISPRa-PP_2329,CRISPRa-PP_4169,CRISPRa-PP_4965,CRISPRa-PP_5079,CRISPRi-PP_0059,CRISPRi-PP_0289,...,CRISPRi-PP_4297,CRISPRi-PP_4678,CRISPRi-PP_4725,CRISPRi-PP_4822,CRISPRi-PP_4823,CRISPRi-PP_5015,CRISPRi-PP_5128,CRISPRi-PP_5155,CRISPRi-PP_5336-5335,pACA production 3 scRNA positive control
genes,,,,,,,,,,,,,,,,,,,,,
AroG-fbr,0.962550,0.914232,0.965946,0.950767,0.857283,0.956012,0.970908,0.947060,0.917213,1.000000,...,0.949286,0.936532,0.879046,0.807352,0.807937,0.939004,0.946881,0.804419,0.808418,0.920279
AroL,1.000000,0.919264,0.975232,0.931820,0.864150,0.935594,0.943711,0.949621,0.948423,0.991431,...,0.960148,0.955710,0.915407,0.821343,0.830362,0.958724,0.942833,0.812715,0.832297,0.911028
MCP-SoxS,0.777355,0.825441,0.900690,0.884231,0.869184,0.862676,0.881482,0.872955,0.912939,1.000000,...,0.871556,0.829625,0.900471,0.838590,0.844952,0.975501,0.864488,0.818373,0.834698,0.872675
PP_0001,0.860293,0.895865,0.859481,0.868216,0.900995,0.866657,0.885766,0.873541,0.935561,1.000000,...,0.851980,0.826100,0.891903,0.780089,0.844089,0.924472,0.792882,0.861916,0.838634,0.905963
PP_0002,0.859410,0.930143,0.881030,0.927069,0.931671,0.867292,0.931671,0.860560,0.967081,1.000000,...,0.928614,0.895626,0.931671,0.836004,0.876925,0.952982,0.887023,0.847492,0.855924,0.948242
PP_0003,0.917585,0.913464,0.978631,0.942476,0.927530,0.826087,0.944277,0.900558,0.972528,0.946057,...,0.893795,0.869385,0.954701,0.802157,0.891474,0.984554,0.898337,0.866764,0.835531,0.949559
PP_0004,0.941891,0.976696,0.954627,0.952912,0.991463,0.954366,0.962628,0.965318,0.998106,0.987607,...,0.975008,0.933299,0.976586,0.908211,0.917777,0.983531,0.963246,0.939709,0.901637,0.989446
PP_0005,1.000000,0.969178,0.964674,0.955683,0.965057,0.971733,0.973186,0.917722,0.936471,0.905537,...,0.903317,0.923727,0.880560,0.912021,0.944440,0.921251,0.922258,0.905537,0.907725,0.980932
PP_0006,0.884557,0.910437,0.877068,0.863265,0.947092,0.913244,0.905932,0.928524,0.965590,1.000000,...,0.929456,0.872467,0.890829,0.872001,0.845383,0.907753,0.869632,0.863265,0.856337,0.911505


In [10]:
# for UREA reaction
cleaned_data.loc['PP_2842':'PP_2849']['CRISPRi-PP_0289']

genes
PP_2842    3.4689
PP_2843    1.9601
PP_2844    0.0953
PP_2845    4.6738
PP_2846    2.0919
PP_2847    2.4069
PP_2848    2.4932
PP_2849    3.5863
Name: CRISPRi-PP_0289, dtype: float64

Use housekeeping genes to determine the variability in measurements
source: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6765009/


In [11]:
import matplotlib.pyplot as plt
import statistics

hkGenes = ['PP_0013', 'PP_5089','PP_0603', 'PP_4121', 'PP_2082', 'PP_1629', 'PP_0447', 'PP_0387']
stdevs = []
for i in hkGenes:
    stdevs.append(statistics.pstdev(norm_data.loc[i].values))

statistics.mean(stdevs)

0.03171450622820654

## troubleshooting

In [12]:
## helper functions
def create_gprdict(model):   
    gpr_dict = dict()
    for rxn in model.reactions:
        if rxn.gene_reaction_rule:
            temp = set()
            for x in [x.strip('() ') for x in rxn.gene_reaction_rule.split(' or ')]:
                temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))
            gpr_dict[rxn.id] = temp
    return gpr_dict

def transcript_value_for_rxn(model, transcriptomics_df, rxn):
    final_transcript_value = 0
    gene_ids = []
    for parallel_gene in create_gprdict(model)[rxn.id]:
        transcript_values = []
        for gene in parallel_gene:
            if gene in transcriptomics_df.index:
                transcript_values.append(transcriptomics_df.loc[gene])
                # transcript_values.append(transcriptomics_df.loc[gene].to_numpy()[0])
#                 print(transcriptomics_df.loc[gene].to_numpy()[0])
            else:
                transcript_values.append(np.inf)
            min_transcript_val = np.min(transcript_values)
        final_transcript_value = final_transcript_value + min_transcript_val
#         if final_transcript_value==newinfbound:
#             display(rxn.id)
#             gene_ids.append(rxn.id)
    return final_transcript_value

In [13]:
Transcriptomics = norm_data['4scRNA control']
model2 = model.copy()
model2.tolerance = 1e-9
sol = model.optimize()
print(sol['RgPAL'])

for rxn in model2.reactions:
    if rxn.id in ['FBA','PGI', 'RgPAL']:
        pass
    elif 'EX_' not in str(rxn.id):
        if rxn.gene_reaction_rule:
            # print(rxn.id)
            if rxn.upper_bound > 0.0:
                rxn.upper_bound = transcript_value_for_rxn(model, Transcriptomics, rxn) * rxn.upper_bound
            else:
                pass
            if rxn.lower_bound < 0.0:
                rxn.lower_bound = transcript_value_for_rxn(model, Transcriptomics, rxn) * rxn.lower_bound
            else:
                pass
                
        else:
            """When there is no GPR, the arbitrary bounds are removed. 
            Common arbitrary bound value of 1000 for E.coli, might be different depending on the model, e.g., 99999.0 for iMM904 yeast model in BiGG"""
            if rxn.lower_bound <= -np.log(1000):
                rxn.lower_bound = -np.Inf
            if rxn.upper_bound >= np.log(1000):
                rxn.upper_bound = np.Inf 

# solve FBA problem with transcriptomic bounds
fba_solution = model2.optimize()
print('FBA status:', fba_solution.status)
print('FBA solution:', fba_solution.objective_value)

Read LP format model from file C:\Users\user\AppData\Local\Temp\tmphe1w4jfb.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
1.5730485607153426
FBA status: optimal
FBA solution: 1.5730485607153426


In [14]:
fba_solution
# for each reaction in the model
for i in model2.reactions:
# if the fba solution is equal to the upper bound, then print reaction id and "upperbound"
    if fba_solution[i.id]==i.upper_bound:
        print(i.id, "upperbound")
# if the fba solution is equal to the lower bound, then print reaction id and "lowerbound"
    if fba_solution[i.id]==i.lower_bound:
        print(i.id, "lowerbound")

BIOMASS_KT2440_WT3 lowerbound
EX_glc__D_e lowerbound
FACOAL161t2pp lowerbound
FBA upperbound
G6PDH2r lowerbound
HACD21i lowerbound
KAT21 lowerbound
LPLIPAL1G161pp lowerbound
LPLIPAL1G181d11pp lowerbound
MLDCP2Bpp lowerbound
MLTGY2pp lowerbound
PLIPA2G180pp lowerbound
PLIPA2G181d11pp lowerbound
AGM4PCPpp lowerbound
ATPM upperbound
ATPM lowerbound
CHRPL lowerbound
EX_4abz_e lowerbound
EX_4aPhe_e lowerbound
PAL_F lowerbound
PAL_Y lowerbound


The biomass, FBA, G6PDH2r, PAL_F, and PAL_Y reactions are constrained so that they do not fall to zero. 
ATPM is constrained to a single literature value. 
Thus, the only reaction that is worth constraining with our transcriptomic data is RgPAL. 

## Eflux2


In [15]:
def runEflux2(Transcriptomics, model):
    model2 = model.copy()
    model2.tolerance = 1e-9
    sol = model.optimize()
    
    for rxn in model2.reactions:
        if rxn.id in ['FBA','PGI', 'RgPAL']:
            pass
        elif 'EX_' not in str(rxn.id):
            if rxn.gene_reaction_rule:
                # print(rxn.id)
                if rxn.upper_bound > 0.0:
                    rxn.upper_bound = transcript_value_for_rxn(model, Transcriptomics, rxn) * rxn.upper_bound
                else:
                    pass
                if rxn.lower_bound < 0.0:
                    rxn.lower_bound = transcript_value_for_rxn(model, Transcriptomics, rxn) * rxn.lower_bound
                else:
                    pass

            else:
                """When there is no GPR, the arbitrary bounds are removed. 
                Common arbitrary bound value of 1000 for E.coli, might be different depending on the model, e.g., 99999.0 for iMM904 yeast model in BiGG"""
                if rxn.lower_bound <= -np.log(1000):
                    rxn.lower_bound = -np.Inf
                if rxn.upper_bound >= np.log(1000):
                    rxn.upper_bound = np.Inf 
    
    model2.reactions.RgPAL.upper_bound = transcript_value_for_rxn(model, Transcriptomics, model.reactions.RgPAL) * sol['RgPAL']
    # solve FBA problem with transcriptomic bounds
    fba_solution = model2.optimize()
    # print('FBA status:', fba_solution.status)
    print('FBA solution:', fba_solution.objective_value)
    
    # constrain the biomass to the optimal value
    for r in model2.reactions:
        if r.objective_coefficient:
            try:
                r.lower_bound = fba_solution.objective_value
                r.upper_bound = fba_solution.objective_value
            except:
                r.upper_bound = fba_solution.objective_value
                r.lower_bound = fba_solution.objective_value

    # Inspect media
#     display(eflux2_model.medium)
    
    # Minimize the sum of squared flux values
    """Note: Because of quadratic objective still have to use cplex objective formulation.
    Optlang does not support quadratic type of constraints and objectives yet."""
    model2.objective = model2.problem.Objective(add([rxn.flux_expression**2 for rxn in model2.reactions]), direction='min')
    
    # solve the minimization of squared fluxes problem
    EFlux2_solution = model2.optimize()
    
    # display(eflux2_model.summary())
    # display(eflux2_model.summary(solution=EFlux2_solution))

    print('E-Flux2 status', EFlux2_solution.status)
    print('E-Flux2 solution', EFlux2_solution.objective_value)
    print()
        
    return EFlux2_solution

In [16]:
fluxes = {}
for strain in norm_data.columns:
    print(strain)
    fluxes[strain] = runEflux2(norm_data[strain], model)

4scRNA control
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpgs6r7cpm.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
FBA solution: 1.5164341231334846
E-Flux2 status optimal
E-Flux2 solution 4443.086473084718

CRISPRa-PP_1075
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp259nn7a9.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
FBA solution: 1.5104954646532736
E-Flux2 status optimal
E-Flux2 solution 4428.3802720676695

CRISPRa-PP_1830
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp9fkalps8.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
FBA solution: 1.5277894876884197
E-Flux2 status optimal
E-Flux2 solution 4471.283158120534

CRISPRa-PP_2168
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp1qmgnpfd.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
FBA solution: 1.506830198872518
E-Flux2 status optimal
E-Flux2 solution 

In [17]:
efluxes = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
efluxes.to_csv('../data/iJN1463_eflux2_flux.csv')

reduced_costs = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['reduced_costs']) for rep in fluxes]))
# reduced_costs.to_csv('core66_eflux2_yield_reduced_costs.csv')


In [18]:
print(efluxes.shape)
efluxes

(637, 41)


,4scRNA control,CRISPRa-PP_1075,CRISPRa-PP_1830,CRISPRa-PP_2168,CRISPRa-PP_2329,CRISPRa-PP_4169,CRISPRa-PP_4965,CRISPRa-PP_5079,CRISPRi-PP_0059,CRISPRi-PP_0289,...,CRISPRi-PP_4297,CRISPRi-PP_4678,CRISPRi-PP_4725,CRISPRi-PP_4822,CRISPRi-PP_4823,CRISPRi-PP_5015,CRISPRi-PP_5128,CRISPRi-PP_5155,CRISPRi-PP_5336-5335,pACA production 3 scRNA positive control
3HAD160,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,...,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419,0.010419
1PQQS,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,...,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065
2PQQS,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,...,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065
3HAACOAT140,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,...,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667,-0.010667
3HAACOAT141,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,...,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667,0.010667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EX_cinm_e,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
R_tyrA,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,...,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751,0.039751
PAL_Y,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
R_pHCA_Tr,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010


## Processing Eflux2 results

In [23]:
efluxResults = efluxes.round(5)

What are the reactions whose flux solutions vary by strain?

In [24]:
varEflux2 = efluxResults[efluxResults.var(axis=1)>1E-5].index

noEX_varEflux2 = [i for i in varEflux2 if 'EX_' not in i]

# check if reaction has a gene reaction rule
noEX_varEflux2 = [i for i in noEX_varEflux2 if model.reactions.get_by_id(i).gene_reaction_rule]
# remove RgPAL
# noEX_varEflux2.remove('RgPAL')

In [25]:
# get the transcriptomics values for reactions listed in noEX_varEflux2
ds = []
for strain in norm_data.columns:
    # get transcriptionFactors for each reaction
    transcriptFactors = [transcript_value_for_rxn(model, norm_data[strain], model.reactions.get_by_id(i)) for i in noEX_varEflux2]
    transcriptFactors = dict(zip(noEX_varEflux2, transcriptFactors))
    # get rid of reactions with infinite transcription
    transcriptFactors = {k: v for k, v in transcriptFactors.items() if v != np.inf}
    ds.append(transcriptFactors)


In [26]:
boundFactors = pd.DataFrame(ds, index=norm_data.columns).transpose()
boundFactors.tail()

strains,4scRNA control,CRISPRa-PP_1075,CRISPRa-PP_1830,CRISPRa-PP_2168,CRISPRa-PP_2329,CRISPRa-PP_4169,CRISPRa-PP_4965,CRISPRa-PP_5079,CRISPRi-PP_0059,CRISPRi-PP_0289,...,CRISPRi-PP_4297,CRISPRi-PP_4678,CRISPRi-PP_4725,CRISPRi-PP_4822,CRISPRi-PP_4823,CRISPRi-PP_5015,CRISPRi-PP_5128,CRISPRi-PP_5155,CRISPRi-PP_5336-5335,pACA production 3 scRNA positive control
DDPA,0.889054,0.914232,0.948481,0.925574,0.857283,0.929645,0.948159,0.927870,0.912316,0.926168,...,0.908321,0.907644,0.879046,0.807352,0.807937,0.873997,0.902806,0.804419,0.808418,0.920279
R_PABASYN_RXN,0.980204,0.957497,0.995524,0.974859,0.933387,0.968044,0.966240,0.979097,0.953145,0.982304,...,0.970946,0.962346,0.967185,0.912694,0.913534,0.945290,0.958328,0.901699,0.915672,1.000000
RXN_20192,0.890601,0.949730,0.978755,0.911726,0.988292,0.911649,0.932567,0.927089,0.948705,0.913962,...,0.928289,0.932839,0.988968,0.980697,0.992602,0.894180,0.912959,0.952851,0.977599,0.973714
RXN_20193,0.985857,0.949793,0.978540,0.929860,0.950992,0.948174,0.936890,0.956916,0.975732,0.976080,...,0.957520,0.951945,1.000000,0.931961,0.926160,0.943306,0.943818,0.918894,0.927636,0.974297
RgPAL,0.964010,0.960234,0.971228,0.957904,0.962712,0.960456,0.965455,0.963376,1.000000,0.995782,...,0.966192,0.959232,0.980158,0.948857,0.936484,0.973278,0.946793,0.936809,0.949307,0.974834


In [27]:
# for each reaction in boundFactors['4scRNA control'], 
for rxn in boundFactors.index:
    model3 = model.copy()
    sol = model3.optimize()
    model3.reactions.RgPAL.upper_bound = sol['RgPAL'] * 0.964010
    print(rxn)
    # multiply the upperbound by the bound Factor
    model3.reactions.get_by_id(rxn).upper_bound = sol[rxn] * boundFactors['4scRNA control'][rxn]
    # run FBA
    model3sol = model3.optimize()
    print('FBA solution:', model3sol.objective_value)


Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpxc6dk5tr.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
ACONTa
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp4ynizvws.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
ACONTb
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp7krtr6zx.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
CS
FBA solution: 1.4679376520626506
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmphco4w2xg.lp
Reading time = 0.02 seconds
: 671 rows, 1274 columns, 5766 nonzeros
CYTBO3_4pp
FBA solution: None


c:\Users\user\miniconda3\envs\new_Gayles\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp1mr25csx.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
DHQS
FBA solution: 1.3418506876585041
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpxy1slevi.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
EDA
FBA solution: 0.5699911577423645
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpncdc1neu.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
EDD
FBA solution: 0.8066169417338137
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp35yj6exh.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
ENO
FBA solution: 1.4983990609196456
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp9sbf5mpm.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
FUM
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpsvfc433r.lp


c:\Users\user\miniconda3\envs\new_Gayles\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp_zjv0iw6.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
NH4tpp
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpjylvglsd.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
PDH
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpup_p02z3.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
PGK
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp6iv92afi.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
PGM
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpkkgcf8z3.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
RPI
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpz5d6yuef.l

c:\Users\user\miniconda3\envs\new_Gayles\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp_06fekpk.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
AKGDH
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpferw6b5d.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
ALCD19
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpru86g0yh.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
ATPS4rpp
FBA solution: None


c:\Users\user\miniconda3\envs\new_Gayles\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Read LP format model from file C:\Users\user\AppData\Local\Temp\tmplhjqq7ri.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
CHORS
FBA solution: 1.4151752108028648
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpiiol6qm9.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
DDPA
FBA solution: 1.385375629423771
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmp0va_o93h.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
R_PABASYN_RXN
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpkiyf7lcl.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
RXN_20192
FBA solution: 1.400957196883513
Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpl48xk8x6.lp
Reading time = 0.00 seconds
: 671 rows, 1274 columns, 5766 nonzeros
RXN_20193
FBA solution: 1.5164345430151993
Read LP format model from file C:\Users\user\AppData\Loca